References:

https://huggingface.co/docs/transformers/model_doc/bert
https://huggingface.co/docs/transformers/index
https://huggingface.co/docs/transformers/custom_datasets

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.2 MB/s eta 0:00:00


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import pickle
import torch
import torch.nn as nn
from tqdm import tqdm
device = torch.device("cuda")


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

In [5]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [6]:
%cd /content/drive/MyDrive/mlops/mlops_course/phase2/reviews

/content/drive/.shortcut-targets-by-id/1pV4N9G9PWwlYVi6Fer0Tq0HFxZXbrdmI/reviews


In [7]:

with open('train.pkl', 'rb') as f:
    train_set = pickle.load(f)
    
with open('test.pkl', 'rb') as f:
    test_set = pickle.load(f)

with open('val.pkl', 'rb') as f:
    val_set = pickle.load(f)         
 

In [8]:
MAX_LEN = 160

In [9]:
from torch.utils.data import Dataset, DataLoader
class GPReviewDataset(Dataset):

  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.reviews)
  
  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]

    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return [encoding['input_ids'].flatten(), encoding['attention_mask'].flatten(),torch.tensor(target, dtype=torch.long)]

In [10]:
df_train = pd.DataFrame.from_dict(train_set)
df_test =  pd.DataFrame.from_dict(test_set)
df_val =  pd.DataFrame.from_dict(val_set)
df_train = df_train.dropna()
df_test = df_test.dropna()
df_val = df_val.dropna()


df_train = df_train.sample(frac=0.01, replace=True, random_state=1)

df_test = df_test.sample(frac=0.01, replace=True, random_state=1)

df_val = df_val.sample(frac=0.01, replace=True, random_state=1)

df_train["y"] = df_train["y"].replace('Fresh','1', regex=True)
df_train["y"] = df_train["y"].replace('Rotten','0', regex=True)
df_train["y"] = df_train["y"].astype(int)


df_test["y"] = df_test["y"].replace('Fresh','1', regex=True)
df_test["y"] = df_test["y"].replace('Rotten','0', regex=True)
df_test["y"] = df_test["y"].astype(int)

df_val["y"] = df_val["y"].replace('Fresh','1', regex=True)
df_val["y"] = df_val["y"].replace('Rotten','0', regex=True)
df_val["y"] = df_val["y"].astype(int)

train_labels = df_train["y"]




In [11]:
df_train.shape, df_val.shape, df_test.shape

((7688, 2), (854, 2), (949, 2))

In [12]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = GPReviewDataset(
    reviews=df.x.to_numpy(),
    targets=df.y.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
  )

In [36]:
BATCH_SIZE = 16
MAX_LEN = 512 
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [38]:
j = 0
for x in train_data_loader :
  j +=1 

print(j)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest s

481


#Load pre-trained BERT model and tokenizer
we proceed with loading the pre-trained BERT model and tokenizer. We would use the tokenizer to convert the text into a format(which has input ids, attention masks) that can be sent to the model.

In [14]:
from transformers import AutoModel, BertTokenizerFast
#load model and tokenizer
bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#Deciding the padding length
If we take the padding length as the maximum length of text found in the training texts, it might leave the training data sparse. Taking the least length would in turn lead to loss of information. Hence, we would plot the graph and see the “average” length and set it as the padding length to trade-off between the two extremes

From the graph above, we take 17 as the padding length.

#Defining the model
We first freeze the BERT pre-trained model, and then add layers as shown in the following code snippets:


https://huggingface.co/docs/transformers/model_doc/bert

In [15]:
# freeze the BERT architecture

for param in bert.parameters():
    param.requires_grad = False

In [39]:
class BERT_architecture(nn.Module):

    def __init__(self, bert):
      
      super(BERT_architecture, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.2)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(768,512)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,2)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)
      
      x = self.fc1(cls_hs)

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      
      # apply softmax activation
      x = self.softmax(x)

      return x
     

In [17]:
# pass the pre-trained BERT to our define architecture
model = BERT_architecture(bert)

# push the model to GPU
model = model.to(device)

In [18]:
#import the AdamW class
from transformers import AdamW
optimizer = AdamW(model.parameters(),lr = 1e-5)  # learning rate

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
#######################################################################

In [19]:
from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_weights = compute_class_weight(class_weight = "balanced",
                                        classes = np.unique(train_labels),
                                        y = train_labels 
                                     )
print("class weights are {} for {}".format(class_weights,np.unique(train_labels)))

class weights are [1.37827178 0.78464993] for [0 1]


In [20]:
#count of both the categories of training labels
pd.value_counts(train_labels)

1    4899
0    2789
Name: y, dtype: int64

#Training the model
After defining the model, define a function to train the model (fine-tune, in this case):

In [21]:
from sklearn.metrics import accuracy_score

In [33]:
#wrap class weights in tensor
weights= torch.tensor(class_weights,dtype=torch.float)

# push weights to GPU
weights = weights.to(device)

# define loss function
# add weights to handle the "imbalance" in the dataset
cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs
epochs = 20

In [43]:
# function to train the model
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step,batch in tqdm(enumerate(train_data_loader)):
    
    # progress update after every 50 batches.
    #if step % 50 == 0 and not step == 0:
      #print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_data_loader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

    

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_data_loader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

Now, define another function that would evaluate the model on validation data.

In [41]:
# function for evaluating the model
def evaluate():
  val_accuracy = []
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []
  val_loss = []

  # iterate over batches
  for step,batch in tqdm(enumerate(val_data_loader)):
    y_true = []
    y_pred = []
    
    # Progress update every 50 batches.
    #if step % 50 == 0 and not step == 0:
      
      # # Calculate elapsed time in minutes.
      # elapsed = format_time(time.time() - t0)
            
      # Report progress.
      #print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_data_loader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      logits = model(sent_id, mask)




    loss = cross_entropy(logits, labels)
    val_loss.append(loss.item())

    preds = torch.argmax(logits, dim=1).flatten()

    accuracy = (preds == labels).cpu().numpy().mean() * 100
    val_accuracy.append(accuracy)
      

    
  #     # model predictions
  #   preds = model(sent_id, mask)

  #     # compute the validation loss between actual and predicted values
  #   loss = cross_entropy(preds,labels)

  #     #y_pred.extend(preds.tolist())
  #   label = labels.detach().cpu().numpy()

  #     #accuracy = (preds == label).cpu().numpy().mean() * 100
  #     #val_accuracy.append(accuracy) 

  #   y_true.extend(label.tolist())
      
      
  #     total_loss = total_loss + loss.item()

  #     preds = preds.detach().cpu().numpy()

  #     total_preds.append(preds)


  #     preds = model(sent_id.to(device), mask.to(device))
  #     preds = preds.detach().cpu().numpy()
  #     preds = np.argmax(preds, axis = 1)
  #     y_pred.extend(preds.tolist())
      
  # #print(preds)
  # #print(labels)
  # # compute the validation loss of the epoch
  # avg_loss = total_loss / len(val_data_loader) 
  # val_acc = accuracy_score(y_true,y_pred)
  # val_accuracy = np.mean(val_accuracy)'''

  # from sklearn.metrics import classification_report
  # pred = np.argmax(preds, axis = 1)
  # print(classification_report(test_y, pred))'''

  # reshape the predictions in form of (number of samples, no. of classes)
  #total_preds  = np.concatenate(total_preds, axis=0)
  val_loss = np.mean(val_loss)
  val_accuracy = np.mean(val_accuracy)
  
  
  return val_loss, total_preds , val_accuracy

In [44]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _ ,val_acc = evaluate()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print('\nTraining Loss: {}'.format(train_loss))
    print('Validation Loss: {}'.format(valid_loss))
    print('Validation accuracy: {}'.format(val_acc))


 Epoch 1 / 20


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s


Evaluating...



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest 


Training Loss: 0.6868987549367417
Validation Loss: 0.6885558664798737
Validation accuracy: 60.7253086419753

 Epoch 2 / 20


...


Evaluating...



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest 


Training Loss: 0.6525340189309229
Validation Loss: 0.6376291414101919
Validation accuracy: 66.62808641975309

 Epoch 14 / 20


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

KeyboardInterrupt: ignored

#Test the data
After fine-tuning the model, test it on the test dataset. Print a classification report to get a better picture of the model’s performance.

In [33]:
# #load weights of best model
# path = 'saved_weights.pt'
# model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
# # get predictions for test data
# with torch.no_grad():
#   preds = model(test_seq.to(device), test_mask.to(device))
#   preds = preds.detach().cpu().numpy()

In [ ]:

# from sklearn.metrics import classification_report

In [ ]:
# pred = np.argmax(preds, axis = 1)
# print(classification_report(test_y, pred))